In [0]:
# Disclaimer: 
# The following code has been excessively commented for educational/demonstration purposes.
# In regular code, please follow a "clean code" approach to commenting instead.

In [0]:
import traceback
from pyspark.sql.functions import col
from configparser import NoSectionError

In [0]:
%run "../utility/util"

In [0]:
%run "../constants/constant"

In [0]:
class PipelineParameter:
  '''
  PipelineParameter holds all parameters passed from ADF pipeline
  '''
  def __init__(self, config_file, pipeline_runtime):
    self.config_file = config_file
    self.pipeline_runtime = pipeline_runtime
    
  def get_config_file(self):
    return self.config_file
  
  def get_pipeline_runtime(self): 
    return self.pipeline_runtime

In [0]:
def write_records_to_bronze_layer(db_params, df, pipeline_runtime):
  '''
  Writes the data to parquet file
  '''
  file_path = create_file_path(Constant.BRONZE_CONTAINER + "/primary_keys", db_params, \
                               pipeline_runtime)
  try:
    file_writer("parquet", df, file_path)
  except Exception:
    traceback.print_exc()
    raise Exception("Error while writing to parquet file for " + file_path)

In [0]:
def get_select_query(db_params, pipeline_params):
  '''
  Returns the select query which needs to be executed on SQL Server
  '''
  primary_keys = get_primary_keys(pipeline_params.get_config_file(), "call_center_tables") \
                            .get(db_params.get_table())
  if len(primary_keys) == 0:
    raise Exception("primary_keys not present for table " + db_params.get_table())
  
  columns = ', '.join(primary_keys)
  return "SELECT " + columns + " FROM " + db_params.get_schema() + "." + db_params.get_table()

In [0]:
def read_records_from_sql_table(db_params, pipeline_params):
  '''
  Reads and returns only primary key columns from SQL Server
  '''
  query = get_select_query(db_params, pipeline_params)
  server = "jdbc:sqlserver://" + db_params.get_server() + ".toromont.com:1433;databaseName="
  try:
    return table_reader_jdbc(server, db_params.get_database(), db_params.get_table(), \
                             db_params.get_user(), db_params.get_password(), "query", query)
  except Exception:
    traceback.print_exc()
    raise Exception("Error while reading from JDBC for " + db_params.get_database() + "." \
                                                         + db_params.get_table())

In [0]:
def load_primary_key_fields_to_bronze_layer(server, tables, pipeline_params):
  '''
  This function copies all of the primary key field data to the bronze layer for 
  later use in determining which keys were deleted
  '''
  password = dbutils.secrets.get(scope = "toromont-kv-secret", key = "datascienceazure")
  for table in tables:
    db_params = get_database_parameters(server, 
                                        table, 
                                        Constant.EXPECTED_TABLE_NAMESPACE_PARTS, 
                                        "datascienceazure", 
                                        password)
    df = read_records_from_sql_table(db_params, pipeline_params)
    write_records_to_bronze_layer(db_params, df, pipeline_params.get_pipeline_runtime())

In [0]:
def create_pipeline_parameters():
  # Read widget values sent from ADF pipeline
  config_file = get_stripped_param("config_file")
  pipeline_runtime = get_stripped_param("pipeline_runtime")
  
  return PipelineParameter(config_file, pipeline_runtime) 

In [0]:
def main():
  # Setting spark properties. Partition specifies no. of partition to create after shuffle operation.
  # This also indicates the no of parallel tasks that spark run to perform operation.
  set_spark_properties(Constant.SPARK_SHUFFLE_PARTITION)
  pipeline_params = create_pipeline_parameters()
  config_file = pipeline_params.get_config_file()
  
  try:
    initial_load = read_config(config_file, "call_center_tables", "initial_load")
    for server, tables in initial_load.items():
      load_primary_key_fields_to_bronze_layer(server, tables, pipeline_params)

  except FileNotFoundError:
    traceback.print_exc()
    raise Exception("Config file " + config_file + " does not exist.")
  except NoSectionError:
    traceback.print_exc()
    raise Exception("Section initial_load does not exist in " + config_file)

In [0]:
main()

Traceback (most recent call last):
 File "<command-2639850298033874>", line 8, in write_records_to_bronze_layer
 file_writer("parquet", df, file_path)
 File "<command-4043258465741107>", line 9, in file_writer
 df.repartition(Constant.PARTITION).write.format(target).mode(save_mode).save(file_path)
 File "/databricks/spark/python/pyspark/sql/dataframe.py", line 838, in repartition
 return DataFrame(self._jdf.repartition(self._jcols(*cols)), self.sql_ctx)
 File "/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
 answer, self.gateway_client, self.target_id, self.name)
 File "/databricks/spark/python/pyspark/sql/utils.py", line 133, in deco
 raise_from(converted)
 File "<string>", line 3, in raise_from
pyspark.sql.utils.AnalysisException: cannot resolve '`8`' given input columns: [InvoiceNumber];;
'RepartitionByExpression ['8], 8
+- Relation[InvoiceNumber#25495] JDBCRelation((SELECT InvoiceNumber FROM dbo.ms_f_invoiceheader_history) SPARK_GEN_SUBQ_6) [numPartitions=1]

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2639850298033874> in write_records_to_bronze_layer (db_params, df, pipeline_runtime) 
 7 try : 
 ----> 8 file_writer ( "parquet" , df , file_path ) 
 9 except Exception : 

 <command-4043258465741107> in file_writer (target, df, file_path, save_mode, part_cols) 
 8 else : 
 ----> 9 df . repartition ( Constant . PARTITION ) . write . format ( target ) . mode ( save_mode ) . save ( file_path ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in repartition (self, numPartitions, *cols) 
 837 cols = ( numPartitions , ) + cols
 --> 838 return DataFrame ( self . _jdf . repartition ( self . _jcols ( * cols ) ) , self . sql_ctx ) 
 839 else : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : cannot resolve '`8`' given input columns: [InvoiceNumber];;
'RepartitionByExpression ['8], 8
+- Relation[InvoiceNumber#25495] JDBCRelation((SELECT InvoiceNumber FROM dbo.ms_f_invoiceheader_history) SPARK_GEN_SUBQ_6) [numPartitions=1]


During handling of the above exception, another exception occurred:

 Exception Traceback (most recent call last)
 <command-2639850298033881> in <module> 
 ----> 1 main ( ) 

 <command-2639850298033880> in main () 
 9 initial_load = read_config ( config_file , "call_center_tables" , "initial_load" ) 
 10 for server , tables in initial_load . items ( ) : 
 ---> 11 load_primary_key_fields_to_bronze_layer ( server , tables , pipeline_params ) 
 12 
 13 except FileNotFoundError : 

 <command-2639850298033878> in load_primary_key_fields_to_bronze_layer (server, tables, pipeline_params) 
 12 password)
 13 df = read_records_from_sql_table ( db_params , pipeline_params ) 
 ---> 14 write_records_to_bronze_layer ( db_params , df , pipeline_params . get_pipeline_runtime ( ) ) 

 <command-2639850298033874> in write_records_to_bronze_layer (db_params, df, pipeline_runtime) 
 9 except Exception : 
 10 traceback . print_exc ( ) 
 ---> 11 raise Exception ( "Error while writing to parquet file for " + file_path ) 

 Exception : Error while writing to parquet file for /mnt/bronze/primary_keys/dwdb/ms_f_invoiceheader_history/2020_11_13/2020_11_13-20_49_11